# EE2213 Project: Multi-Class Classification

#### <span style="color:red">No additional library imports are permitted.</span>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder

## PART 0: Data Loading and Preparation

In [4]:
def load_openml_dataset():

    dataset = fetch_openml(name='vehicle', version=1, as_frame=True, parser='auto')
    X = dataset.data.values
    
    target_values = dataset.target.values
    unique_targets = np.unique(target_values)
    
    # Create mapping from string labels to integers
    label_to_int = {label: i for i, label in enumerate(unique_targets)}
    y = np.array([label_to_int[label] for label in target_values])
    
    feature_names = dataset.feature_names
    
    return X, y, feature_names

X, y, feature_names = load_openml_dataset()
print(f"Feature shape: {X.shape}, target output shape:{y.shape}")
print(f"feature names: {feature_names}")


Feature shape: (846, 18), target output shape:(846,)
feature names: ['COMPACTNESS', 'CIRCULARITY', 'DISTANCE_CIRCULARITY', 'RADIUS_RATIO', 'PR.AXIS_ASPECT_RATIO', 'MAX.LENGTH_ASPECT_RATIO', 'SCATTER_RATIO', 'ELONGATEDNESS', 'PR.AXIS_RECTANGULARITY', 'MAX.LENGTH_RECTANGULARITY', 'SCALED_VARIANCE_MAJOR', 'SCALED_VARIANCE_MINOR', 'SCALED_RADIUS_OF_GYRATION', 'SKEWNESS_ABOUT_MAJOR', 'SKEWNESS_ABOUT_MINOR', 'KURTOSIS_ABOUT_MAJOR', 'KURTOSIS_ABOUT_MINOR', 'HOLLOWS_RATIO']


## PART 1: Dataset Partition and One-hot Encoding

In [5]:
def dataset_partition_encoding(X, y):
    """
    Input type
    :X type: numpy.ndarray of size (number_of_samples, number_of_features)
    :y type: numpy.ndarray of size (number_of_samples,)

    Return type
    :X_train type: numpy.ndarray of size (number_of_training_samples, number_of_features)
    :X_val type: numpy.ndarray of size (number_of_validation_samples, number_of_features)
    :X_test type: numpy.ndarray of size (number_of_test_samples, number_of_features)
    :Ytr_onehot type: numpy.ndarray of size (number_of_training_samples, num_classes)
    :Yval_onehot type: numpy.ndarray of size (number_of_validation_samples, num_classes)
    :Yts_onehot type: numpy.ndarray of size (number_of_test_samples, num_classes)

    """

    # your code goes here
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=665, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=665, stratify=y_temp)
    
    encoder = OneHotEncoder(sparse_output=False)
    Ytr_onehot = encoder.fit_transform(y_train.reshape(-1, 1))
    Yval_onehot = encoder.transform(y_val.reshape(-1, 1))
    Yts_onehot = encoder.transform(y_test.reshape(-1, 1))
    

    # return in this order
    return X_train, X_val, X_test, Ytr_onehot, Yval_onehot, Yts_onehot

X_train, X_val, X_test, Ytr_onehot, Yval_onehot, Yts_onehot = dataset_partition_encoding(X, y)
print(f"Training set shape: {X_train.shape}, {Ytr_onehot.shape}")
print(f"Validation set shape: {X_val.shape}, {Yval_onehot.shape}")
print(f"Test set shape: {X_test.shape}, {Yts_onehot.shape}")

Training set shape: (507, 18), (507, 4)
Validation set shape: (169, 18), (169, 4)
Test set shape: (170, 18), (170, 4)


## PART 2: Feature Selection using Pearson Correlation

In [6]:
def feature_selection(X_train, X_val, X_test, feature_names, threshold=0.8):
    """
    Input type
    :X_train type: numpy.ndarray of size (number_of_training_samples, number_of_features)
    :X_val type: numpy.ndarray of size (number_of_validation_samples, number_of_features)
    :X_test type: numpy.ndarray of size (number_of_test_samples, number_of_features)
    :feature_names type: list of str
    :threshold type: float

    Return type
    :selected_features type: list of str
    :FS_X_train type: numpy.ndarray of size (number_of_training_samples, number_of_selected_features)
    :FS_X_val type: numpy.ndarray of size (number_of_validation_samples, number_of_selected_features)
    :FS_X_test type: numpy.ndarray of size (number_of_test_samples, number_of_selected_features)

    """

    # your code goes here
    corr_matrix = np.corrcoef(X_train, rowvar=False)

    to_keep = [0]

    for i in range(1, X_train.shape[1]):
        is_correlated = False
        for j in to_keep:
            if abs(corr_matrix[i, j]) > threshold:
                is_correlated = True
                break
        if not is_correlated:
            to_keep.append(i)

    selected_features = [feature_names[i] for i in to_keep]
    FS_X_train = X_train[:, to_keep]
    FS_X_val = X_val[:, to_keep]
    FS_X_test = X_test[:, to_keep]

    # return in this order
    return selected_features, FS_X_train, FS_X_val, FS_X_test


selected_features, FS_X_train, FS_X_val, FS_X_test = feature_selection(X_train, X_val, X_test, feature_names)


print(f"{len(selected_features)} Selected Features: {selected_features}")
print(f"Training set shape after feature selection: {FS_X_train.shape}, {Ytr_onehot.shape}")
print(f"Validation set shape after feature selection: {FS_X_val.shape}, {Yval_onehot.shape}")
print(f"Test set shape after feature selection: {FS_X_test.shape}, {Yts_onehot.shape}")


10 Selected Features: ['COMPACTNESS', 'CIRCULARITY', 'DISTANCE_CIRCULARITY', 'RADIUS_RATIO', 'PR.AXIS_ASPECT_RATIO', 'MAX.LENGTH_ASPECT_RATIO', 'SKEWNESS_ABOUT_MAJOR', 'SKEWNESS_ABOUT_MINOR', 'KURTOSIS_ABOUT_MAJOR', 'KURTOSIS_ABOUT_MINOR']
Training set shape after feature selection: (507, 10), (507, 4)
Validation set shape after feature selection: (169, 10), (169, 4)
Test set shape after feature selection: (170, 10), (170, 4)


## PART 3: Polynomial Feature Transformation and Classification

In [ ]:
def polynomial_for_classification(FS_X_train, FS_X_val, FS_X_test, Ytr_onehot, Yval_onehot, Yts_onehot, max_order=3, lamda=0.001):
    """
    Args:
        FS_X_train (np.ndarray): Feature matrix for training.
        FS_X_val (np.ndarray): Feature matrix for validation.
        FS_X_test (np.ndarray): Feature matrix for testing.
        Ytr_onehot (np.ndarray): One-hot encoded labels for training.
        Yval_onehot (np.ndarray): One-hot encoded labels for validation.
        Yts_onehot (np.ndarray): One-hot encoded labels for testing.
        max_order (int): Maximum polynomial order to consider.
        lamda (float): Regularization strength.

    Returns:
        acc_train_list (list): Training accuracies for each polynomial order.
        acc_val_list (list): Validation accuracies for each polynomial order.
        best_order (int): Best polynomial order based on validation accuracy.
        acc_test (float): Test accuracy for the best polynomial order.

    """

    # your code goes here
    acc_train_list = []
    acc_val_list = []
    acc_test = 0.0
    best_order = None
    best_W = None

    for order in range(1, 4):
        features = PolynomialFeatures(degree=order)
        poly_tr = features.fit_transform(FS_X_train)
        poly_val = features.transform(FS_X_val)
        
        r, c = poly_tr.shape

        if r > c:
            XtX = poly_tr.T @ poly_tr
            try:
                W = np.linalg.inv(XtX) @ poly_tr.T @ Ytr_onehot
            except np.linalg.LinAlgError:
                W = np.linalg.inv(XtX + lamda * np.eye(c)) @ poly_tr.T @ Ytr_onehot

        elif r < c:
            XXt = poly_tr @ poly_tr.T
            try:
                W = poly_tr.T @ np.linalg.inv(XXt) @ Ytr_onehot
            except np.linalg.LinAlgError:
                W = poly_tr.T @ np.linalg.inv(XXt + lamda * np.eye(r)) @ Ytr_onehot

        Ytr_pred = np.argmax(poly_tr @ W, axis=1)
        Yval_pred = np.argmax(poly_val @ W, axis=1)

        Ytr_true = np.argmax(Ytr_onehot, axis=1)
        Yval_true = np.argmax(Yval_onehot, axis=1)

        acc_train_list.append(accuracy_score(Ytr_true, Ytr_pred))
        acc_val_list.append(accuracy_score(Yval_true, Yval_pred))

        if best_order is None or acc_val_list[-1] > acc_val_list[-2]:
            best_order = order
            best_W = W

    best = PolynomialFeatures(degree=best_order)
    poly_test = best.fit_transform(FS_X_test)
    Yts_pred = np.argmax(poly_test @ best_W, axis=1)
    Yts_true = np.argmax(Yts_onehot, axis=1)

    acc_test = accuracy_score(Yts_true, Yts_pred)

    # return in this order              
    return acc_train_list, acc_val_list, best_order, acc_test

acc_train_list, acc_val_list, best_order, acc_test = polynomial_for_classification(FS_X_train, FS_X_val, FS_X_test, Ytr_onehot, Yval_onehot, Yts_onehot)

print(f"Training accuracies: {np.round(acc_train_list,2)}")
print(f"Validation accuracies: {np.round(acc_val_list,2)}")
print(f"Best polynomial order: {best_order}")
print(f"Test accuracy for best order {best_order}: {np.round(acc_test,2)}")


Training accuracies: [0.72 0.88 0.96]
Validation accuracies: [0.71 0.74 0.72]
Best polynomial order: 2
Test accuracy for best order 2: 0.91


## PART 4: Multinomial Logistic Regression

In [8]:
def MLR_select_lr(FS_X_train, FS_X_val, FS_X_test, Ytr_onehot, Yval_onehot, Yts_onehot, lr_list=[0.0001, 0.001, 0.01, 0.1], num_iters=20000):
    """
    Args:
        FS_X_train (np.ndarray): Feature matrix for training.
        FS_X_val (np.ndarray): Feature matrix for validation.
        FS_X_test (np.ndarray): Feature matrix for testing.
        Ytr_onehot (np.ndarray): One-hot encoded labels for training.
        Yval_onehot (np.ndarray): One-hot encoded labels for validation.
        Yts_onehot (np.ndarray): One-hot encoded labels for testing.
        lr_list (list): List of learning rates to test.
        num_iters (int): Number of iterations for training.

    Returns:
        cost_dict (dict): Dictionary of cost values for each learning rate without input normalization.
                          example: cost_dict = {0.0001: [0.1, 0.05, ...], 0.001: [0.09, 0.045, ...], ...}
        acc_train_list_Log (list): Training accuracies for each learning rate without input normalization.
        acc_val_list_Log (list): Validation accuracies for each learning rate without input normalization.
        best_lr (float): Best learning rate based on validation accuracy without input normalization.
        acc_test (float): Test accuracy for the best learning rate without input normalization.
        cost_dict_norm (dict): Dictionary of cost values for each learning rate with input normalization.
        acc_train_list_Log_norm (list): Training accuracies for each learning rate with input normalization.
        acc_val_list_Log_norm (list): Validation accuracies for each learning rate with input normalization.
        best_lr_norm (float): Best learning rate based on validation accuracy with input normalization.
        acc_test_norm (float): Test accuracy for the best learning rate with input normalization.

    """
    
    # your code goes here


    # return in this order      

    return cost_dict,acc_train_list_Log, acc_val_list_Log, best_lr,test_acc_Log, cost_dict_norm, acc_train_list_Log_norm, acc_val_list_Log_norm, best_lr_norm, test_acc_Log_norm

def cost_vs_iter_curve (cost_dict, cost_dict_norm):
     """
    Args:
        cost_dict (dict): Dictionary of cost values for each learning rate without input normalization.
        cost_dict_norm (dict): Dictionary of cost values for each learning rate with input normalization.

    """

    # your code goes here



cost_dict,acc_train_list_Log, acc_val_list_Log, best_lr,test_acc_Log, cost_dict_norm, acc_train_list_Log_norm, acc_val_list_Log_norm, best_lr_norm, test_acc_Log_norm = MLR_select_lr(FS_X_train, FS_X_val, FS_X_test, Ytr_onehot, Yval_onehot, Yts_onehot)

print(f"Without Normalization")
print(f"Training accuracies for different learning rates: {np.round(acc_train_list_Log,2)}")
print(f"Validation accuracies for different learning rates: {np.round(acc_val_list_Log,2)}")
print(f"Best learning rate: {best_lr}")
print(f"Test accuracy for best learning rate {best_lr}: {np.round(test_acc_Log,2)}")


print(f"With Z-score Standardization")
print(f"Training accuracies for different learning rates: {np.round(acc_train_list_Log_norm,2)}")
print(f"Validation accuracies for different learning rates: {np.round(acc_val_list_Log_norm,2)}")
print(f"Best learning rate: {best_lr_norm}")
print(f"Test accuracy for best learning rate {best_lr_norm}: {np.round(test_acc_Log_norm,2)}")

cost_vs_iter_curve (cost_dict, cost_dict_norm)

NameError: name 'cost_dict' is not defined

### Analysis of Effect of Normalization Based on Your Results

#### Add Markdown cell below to analyze the effect of normalization based on your results
<span style="color:orange">(delete this markdown cell before submission)</span>